In [1]:
from datetime import datetime as dt
from collections import defaultdict
import time
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
import matplotlib.pyplot as plt
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize(path="C:/MetaTrader_A/MetaTrader 5/terminal64.exe",login=1049732, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
def MetaLink(p,x,y,z):
    if x == "D1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_D1, z, y)
    elif x == "H8":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H8, z, y)
    elif x == "H4":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H4, z, y)
    elif x == "H3":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H3, z, y)
    elif x == "H2":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H2, z, y)
    elif x == "H1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H1, z, y)
    elif x == "M30":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M30, z, y)
    elif x == "M15":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M15, z, y)
    elif x == "M5":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M5, z, y)
    elif x == "M1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M1, z, y)
    return data

In [4]:
def MetaLink_range(p,x,y,z):
    if x == "D1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_D1, y, z)
    elif x == "H8":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H8, y, z)
    elif x == "H4":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H4, y, z)
    elif x == "H3":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H3, y, z)
    elif x == "H2":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H2, y, z)
    elif x == "H1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H1, y, z)
    elif x == "M30":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M30, y, z)
    elif x == "M15":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M15, y, z)
    elif x == "M5":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M5, y, z)
    elif x == "M1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M1, y, z)
    return data

In [5]:
def MetaLink_pos(p,x,y):
    if x == "D1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_D1, 0, y)
    elif x == "H8":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H8, 0, y)
    elif x == "H4":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H4, 0, y)
    elif x == "H3":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H3, 0, y)
    elif x == "H2":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H2, 0, y)
    elif x == "H1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H1, 0, y)
    elif x == "M30":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M30, 0, y)
    elif x == "M15":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M15, 0, y)
    elif x == "M5":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M5, 0, y)
    elif x == "M1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M1, 0, y)
    return data

In [6]:
def levelgrab(pair):
    
    levslist = []
    
    for x in pair:
        Srates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H1, 0, 1000)
        S = Srates
        SCM = np.average(abs(S['close'] - S['open']))
        y = S['close']
        LVL = np.concatenate((y[argrelextrema(y, np.greater)[0]],y[argrelextrema(y, np.less)[0]]))
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = [(Ba[i], Bb[i]) for i in range(0, len(Ba))] 
        Toutput = defaultdict(int) 
        for k, v in T: 
            Toutput[k] += v 
        table = list(Toutput.items())
        tablevs = [x for x, y in table]
        tabpings = [y for x, y in table]
        tabmax = argrelextrema(np.array(tabpings), np.greater)
        tabmax2 = list(tabmax[0])
        levs = np.array(tablevs)[tabmax2]
        levs = list(levs)
        levs.sort(reverse=True)
        levs = list(levs)
        
        mindist = min([abs(j-i) for i, j in zip(levs[:-1], levs[1:])])
        
        while mindist < SCM:
            i = 0
            h = -1
            j = 1
            for c in range(0, len(levs)):
                if j == len(levs):
                    j = -1
                if abs(levs[i]-levs[h]) < SCM:
                    levs[i] = 0
                elif abs(levs[i]-levs[j]) < SCM:
                    levs[i] = levs[i] - abs(levs[i]-levs[j])/2
                i += 1
                h += 1
                j += 1
            levs = [v for v in levs if v != 0]
            mindist = min([abs(a-b) for a, b in zip(levs[:-1], levs[1:])])
            
        levs = tuple(levs)

        levslist.append(levs)

    return levslist

In [7]:
def Pip_Screen(a,d):
    
    base, quote = a[:3], a[3:]

    if quote == "USD":
        exchange = 1.00
        if base == "XAG":
            D = d*10000
        elif base == "XAU":
            D = d*100
        elif base == "BTC":
            D = d*10
        elif base == "BCH":
            D = d*100
        elif base == "ETH":
            D = d*10
        elif base == "DSH":
            D = d*100
        elif base == "LTC":
            D = d*100
        elif base == "XMR":
            D = d*100
        elif base == "ZEC":
            D = d*100
        elif base == "ETC":
            D = d*1000
        elif base == "XRP":
            D = d*10000
        else:
            D = d*10000
    if quote == "USDt":
        exchange = 1.00
        if base == "BTC":
            D = d*10
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
        D = d*10000
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
        D = d*10000
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
        D = d*10000
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
        D = d*10000
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
        D = d*10000
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
        D = d*100
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
        D = d*10000
    elif quote == 'NOK':
        exchange = mt5.symbol_info("USDNOK").bid
        D = d*1000
    elif quote == 'SEK':
        exchange = mt5.symbol_info("USDSEK").bid
        D = d*1000
    elif quote == 'SGD':
        exchange = mt5.symbol_info("USDSGD").bid
        D = d*10000
    elif quote == 'DKK':
        exchange = mt5.symbol_info("USDDKK").bid
        D = d*1000
    elif quote == 'MXN':
        exchange = mt5.symbol_info("USDMXN").bid
        D = d*1000
    elif quote == "entCrude":
        exchange = 1.00
        D = d*10
    elif quote == "ICrude":
        exchange = 1.00
        D = d*10
    elif quote == "TRY":
        exchange = mt5.symbol_info("USDTRY").bid
        D = d*10000
        
    return D
    

In [8]:
def Pip_ReverseScreen(a,d):
    
    base, quote = a[:3], a[3:]

    if quote == "USD":
        if base == "XAG":
            D = d/10000
        elif base == "XAU":
            D = d/100
        elif base == "BTC":
            D = d/10
        elif base == "BCH":
            D = d/100
        elif base == "ETH":
            D = d/10
        elif base == "DSH":
            D = d/100
        elif base == "LTC":
            D = d/100
        elif base == "XMR":
            D = d/100
        elif base == "ZEC":
            D = d/100
        elif base == "ETC":
            D = d/1000
        elif base == "XRP":
            D = d/10000
        else:
            D = d/10000
    if quote == "USDt":
        if base == "BTC":
            D = d/10
    elif quote == "AUD":
        D = d/10000
    elif quote == "CAD":
        D = d/10000
    elif quote == "CHF":
        D = d/10000
    elif quote == "EUR": 
        D = d/10000
    elif quote == "GBP":
        D = d/10000
    elif quote == "JPY":
        D = d/100
    elif quote == "NZD":
        D = d/10000
    elif quote == 'NOK':
        D = d/1000
    elif quote == 'SEK':
        D = d/1000
    elif quote == 'SGD':
        D = d/10000
    elif quote == 'DKK':
        D = d/1000
    elif quote == 'MXN':
        D = d/1000
    elif quote == "entCrude":
        exchange = 1.00
        D = d/10
    elif quote == "ICrude":
        exchange = 1.00
        D = d/10
    elif quote == "TRY":
        D = d/10000
        
    return D

In [9]:
def positioner(a,b,c,d,e):
    
    riskusd = 50.00        

    base, quote = a[:3], a[3:]
    
    ro = 2

    if quote == "USD":
        exchange = 1.00
        if base == "XAG":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        elif base == "XAU":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BTC":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BCH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETH":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "DSH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "LTC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "XMR":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ZEC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETC":
            stpips = abs(e - c)*1000
            lot = round(((riskusd*exchange)/10)/10/stpips,ro)
        elif base == "XRP":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        else:
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/stpips,ro)
    if quote == "USDt":
        exchange = 1.00
        if base == "BTC":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
        stpips = abs(e - c)*100
        lot = round(((riskusd*exchange)/10)/100/stpips,ro)
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'NOK':
        exchange = mt5.symbol_info("USDNOK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SEK':
        exchange = mt5.symbol_info("USDSEK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SGD':
        exchange = mt5.symbol_info("USDSGD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'DKK':
        exchange = mt5.symbol_info("USDDKK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'MXN':
        exchange = mt5.symbol_info("USDMXN").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == "entCrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "ICrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "TRY":
        exchange = mt5.symbol_info("USDTRY").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
        
    deviation = 20
    
    if b == "LONG":
        price = mt5.symbol_info_tick(a).ask
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "RUBY",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
    elif b == "SHORT":
        price = mt5.symbol_info_tick(a).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "RUBY",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
            
    lotflag = "GREEN"
    if lot == 0:
        lotflag = "RED"
    
    return request, lotflag

In [10]:
def RUBY(pair,levslist):
    
    ia = 0
    for p in pair:
    
        G = MetaLink_pos(p,"M5",50)
        Gy = Pip_Screen(p,G['close'])
        Gx = range(0,len(Gy))
        slope, intercept, r_value, p_value, std_err = stats.linregress(Gx,Gy)
        ST_slope = slope
        scopetheta = abs(math.degrees(math.atan(ST_slope)))
        if scopetheta > 30:

            D = G[-8:-1]

            try:
                PSTICK = abs((D['close'] - D['open'])/(D['high'] - D['low']))
            except RuntimeWarning:
                PSTICK = 0
                
            ANIMAL = []
            GAUGE = []
            READING = []
            READING2 = []
            i = 0
            for t in range(0,len(PSTICK)): 
                if PSTICK[i] <= 1.1:
                    if PSTICK[i] < 0.85:
                        if PSTICK[i] < 0.65:
                            if PSTICK[i] < 0.50:
                                if PSTICK[i] <= 0.30:
                                    if PSTICK[i] <= 0.15:
                                        animal = 'Doji'
                                    else:
                                        animal = 'Frog'
                                else:
                                    animal = 'Cat'
                            else:
                                animal = 'Dog'
                        else:
                            animal = 'Horse'
                    else:
                        animal = 'Shark'

                ANIMAL.append(animal)

                cndmid = (D['close'][i]+D['open'][i])/2
                stickmid = (D['high'][i]+D['low'][i])/2

                if (D['high'][i] - D['low'][i]) == 0:
                    midpoint = D['close'][i]
                else:
                    midpoint = (cndmid - stickmid)/(D['high'][i] - D['low'][i])

                if midpoint >= 0.075:
                    if midpoint >= 0.175:
                        if midpoint >= 0.25:
                            gauge = 'Top Saturated'
                        else:
                            gauge = 'Top Stacked'
                    else:
                        gauge = 'Top Semi-Stacked'
                elif midpoint <= -0.075:
                    if midpoint <= -0.175:
                        if midpoint <= -0.25:
                            gauge = 'Bottom Saturated'
                        else:
                            gauge = 'Bottom Stacked'
                    else:
                        gauge = 'Bottom Semi-Stacked'
                else:
                    gauge = 'Centered'

                GAUGE.append(gauge)

                if ST_slope < 0:
                    if GAUGE[i] == 'Top Saturated':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'JACK'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'DIPPER'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'HAMMER'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'DRAGONFLY DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Top Stacked':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'JACK'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'SKILLET'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'MALLET'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'DRAGONFLY DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Top Semi-Stacked':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'PLUMP DOJI'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'LONG-LEG DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Centered':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'PLUMP DOJI'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'LONG-LEG DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Bottom Semi-Stacked':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'PLUMP DOJI'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'LONG-LEG DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Bottom Stacked':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'BOOST'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'BRAKE'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'BRAKE'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Bottom Saturated':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'BOOST'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'BRAKE'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'BRAKE'
                        else:
                            Read = ANIMAL[i]
                    else:
                        Read = ANIMAL[i]
                elif ST_slope > 0:
                    if GAUGE[i] == 'Bottom Saturated':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'JACK'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'DIPPER'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'SHOOTING STAR'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'GRAVESTONE DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Bottom Stacked':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'JACK'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'SKILLET'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'ASTEROID'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'GRAVESTONE DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Bottom Semi-Stacked':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'PLUMP DOJI'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'LONG-LEG DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Centered':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'PLUMP DOJI'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'LONG-LEG DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Top Semi-Stacked':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'PLUMP DOJI'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'LONG-LEG DOJI'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Top Stacked':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'BOOST'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'BRAKE'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'BRAKE'
                        else:
                            Read = ANIMAL[i]
                    elif GAUGE[i] == 'Top Saturated':
                        if ANIMAL[i] == 'Shark':
                            Read = 'MARIBOZU'
                        elif ANIMAL[i] == 'Horse':
                            Read = 'TUBE'
                        elif ANIMAL[i] == 'Dog':
                            Read = 'BOOST'
                        elif ANIMAL[i] == 'Cat':
                            Read = 'GLIDE'
                        elif ANIMAL[i] == 'Frog':
                            Read = 'BRAKE'
                        elif ANIMAL[i] == 'Doji':
                            Read = 'BRAKE'
                        else:
                            Read = ANIMAL[i]
                    else:
                        Read = ANIMAL[i]
                else:
                    Read = ANIMAL[i]
                READING.append(Read)

                if i == 0:
                    READING2.append(0)
                else:
                    ii = i - 1
                    if ST_slope < 0:
                        if (D['close'][ii] - D['open'][ii]) < 0:
                            if D['close'][i] > D['high'][ii]:
                                if READING[ii] == 'BRAKE':
                                    if (D['close'][i] - D['open'][i]) > 0:
                                        if READING[i] == 'MARIBOZU':
                                            Read2 = 'BULLISH ENGULFING'
                                        elif READING[i] == 'TUBE':
                                            Read2 = 'BULLISH ENGULFING'
                                        else:
                                            Read2 = 0
                                    else:
                                        Read2 = 0
                                elif READING[ii] == 'GLIDE':
                                    if (D['close'][i] - D['open'][i]) > 0:
                                        if READING[i] == 'MARIBOZU':
                                            Read2 = 'BULLISH ENGULFING'
                                        elif READING[i] == 'TUBE':
                                            Read2 = 'BULLISH ENGULFING'
                                        else:
                                            Read2 = 0
                                    else:
                                        Read2 = 0
                                elif READING[ii] == 'PLUMP DOJI':
                                    if (D['close'][i] - D['open'][i]) > 0:
                                        if READING[i] == 'MARIBOZU':
                                            Read2 = 'BULLISH ENGULFING'
                                        elif READING[i] == 'TUBE':
                                            Read2 = 'BULLISH ENGULFING'
                                        else:
                                            Read2 = 0
                                    else:
                                        Read2 = 0
                                elif READING[ii] == 'LONG-LEG DOJI':
                                    if (D['close'][i] - D['open'][i]) > 0:
                                        if READING[i] == 'MARIBOZU':
                                            Read2 = 'BULLISH ENGULFING'
                                        elif READING[i] == 'TUBE':
                                            Read2 = 'BULLISH ENGULFING'
                                        else:
                                            Read2 = 0
                                    else:
                                        Read2 = 0
                                else:
                                    Read2 = 0
                            elif D['open'][i] == D['close'][ii]:
                                if D['low'][i] == D['low'][ii]:
                                    if (D['close'][i] - D['open'][i]) > 0:
                                        if D['close'][i] > D['open'][ii]:
                                            Read2 = 'TWEEZERS BOTTOM'
                                        else:
                                            Read2 = 0
                                    else:
                                        Read2 = 0
                                else:
                                    Read2 = 0
                            elif D['close'][i] > (D['open'][ii]+D['close'][ii])/2:
                                if D['open'][i] < D['close'][ii]:
                                    if (D['close'][ii] - D['open'][ii]) > 0:
                                        Read2 = 'PIERCING PATTERN'
                                    else:
                                        Read2 = 0
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    elif ST_slope > 0:
                        if (D['close'][ii] - D['open'][ii]) > 0:
                            if D['close'][i] < D['low'][ii]:
                                if READING[ii] == 'BRAKE':
                                    if (D['close'][i] - D['open'][i]) < 0:
                                        if READING[i] == 'MARIBOZU':
                                            Read2 = 'BEARISH ENGULFING'
                                        elif READING[i] == 'TUBE':
                                            Read2 = 'BEARISH ENGULFING'
                                        else:
                                            Read2 = 0
                                    else:
                                        Read2 = 0
                                elif READING[ii] == 'GLIDE':
                                    if (D['close'][i] - D['open'][i]) < 0:
                                        if READING[i] == 'MARIBOZU':
                                            Read2 = 'BEARISH ENGULFING'
                                        elif READING[i] == 'TUBE':
                                            Read2 = 'BEARISH ENGULFING'
                                        else:
                                            Read2 = 0
                                    else:
                                        Read2 = 0
                                elif READING[ii] == 'PLUMP DOJI':
                                    if (D['close'][i] - D['open'][i]) < 0:
                                        if READING[i] == 'MARIBOZU':
                                            Read2 = 'BEARISH ENGULFING'
                                        elif READING[i] == 'TUBE':
                                            Read2 = 'BEARISH ENGULFING'
                                        else:
                                            Read2 = 0
                                    else:
                                        Read2 = 0
                                elif READING[ii] == 'LONG-LEG DOJI':
                                    if (D['close'][i] - D['open'][i]) < 0:
                                        if READING[i] == 'MARIBOZU':
                                            Read2 = 'BEARISH ENGULFING'
                                        elif READING[i] == 'TUBE':
                                            Read2 = 'BEARISH ENGULFING'
                                        else:
                                            Read2 = 0
                                    else:
                                        Read2 = 0
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D['close'][i] == D['open'][ii]:
                            if D['high'][i] == D['high'][ii]:
                                if (D['close'][i] - D['open'][i]) < 0:
                                    if D['close'][i] < D['open'][ii]:
                                        Read2 = 'TWEEZERS TOP'
                                    else:
                                        Read2 = 0
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D['close'][i] < (D['open'][ii]+D['close'][ii])/2:
                            if D['open'][i] > D['close'][ii]:
                                if (D['close'][ii] - D['open'][ii]) < 0:
                                    Read2 = 'DARK CLOUD COVER'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                    READING2.append(Read2)

                i += 1

            i1 = -1
            i2 = -2
            i3 = -3  
            i4 = -4 
            i5 = -5  
            if (D['close'][i5] - D['open'][i5]) > 0:
                if READING[i5] == 'MARIBOZU':
                    if (D['close'][i4] - D['open'][i4]) < 0:
                        if D['close'][i4] > D['open'][i5]:
                            if (D['close'][i3] - D['open'][i3]) < 0:
                                if D['close'][i3] > D['open'][i5]:
                                    if (D['close'][i2] - D['open'][i2]) < 0:
                                        if D['close'][i2] > D['open'][i5]:
                                            if (D['close'][i1] - D['open'][i1]) > 0:
                                                if READING[i1] == 'MARIBOZU':
                                                    Read3 = 'RISING THREE'
                                                elif READING[i1] == 'TUBE':
                                                    Read3 = 'RISING THREE'
                                                else:
                                                    Read3 = 0
                                            else:
                                                Read3 = 0
                                        else:
                                            Read3 = 0
                                    else:
                                        Read3 = 0
                                else:
                                    Read3 = 0                               
                            else:
                                Read3 = 0
                        else:
                            Read3 = 0
                    else:
                        Read3 = 0
                elif READING[i5] == 'TUBE':
                    if (D['close'][i4] - D['open'][i4]) < 0:
                        if D['close'][i4] > D['open'][i5]:
                            if (D['close'][i3] - D['open'][i3]) < 0:
                                if D['close'][i3] > D['open'][i5]:
                                    if (D['close'][i2] - D['open'][i2]) < 0:
                                        if D['close'][i2] > D['open'][i5]:
                                            if (D['close'][i1] - D['open'][i1]) > 0:
                                                if READING[i1] == 'MARIBOZU':
                                                    Read3 = 'RISING THREE'
                                                elif READING[i1] == 'TUBE':
                                                    Read3 = 'RISING THREE'
                                                else:
                                                    Read3 = 0
                                            else:
                                                Read3 = 0
                                        else:
                                            Read3 = 0
                                    else:
                                        Read3 = 0
                                else:
                                    Read3 = 0                               
                            else:
                                Read3 = 0
                        else:
                            Read3 = 0
                    else:
                        Read3 = 0
                else:
                    Read3 = 0
            elif (D['close'][i5] - D['open'][i5]) < 0:
                if READING[i5] == 'MARIBOZU':
                    if (D['close'][i4] - D['open'][i4]) > 0:
                        if D['close'][i4] < D['open'][i5]:
                            if (D['close'][i3] - D['open'][i3]) > 0:
                                if D['close'][i3] < D['open'][i5]:
                                    if (D['close'][i2] - D['open'][i2]) > 0:
                                        if D['close'][i2] < D['open'][i5]:
                                            if (D['close'][i1] - D['open'][i1]) < 0:
                                                if READING[i1] == 'MARIBOZU':
                                                    Read3 = 'FALLING THREE'
                                                elif READING[i1] == 'TUBE':
                                                    Read3 = 'FALLING THREE'
                                                else:
                                                    Read3 = 0
                                            else:
                                                Read3 = 0
                                        else:
                                            Read3 = 0
                                    else:
                                        Read3 = 0
                                else:
                                    Read3 = 0                               
                            else:
                                Read3 = 0
                        else:
                            Read3 = 0
                    else:
                        Read3 = 0
                elif READING[i5] == 'TUBE':
                    if (D['close'][i4] - D['open'][i4]) > 0:
                        if D['close'][i4] < D['open'][i5]:
                            if (D['close'][i3] - D['open'][i3]) > 0:
                                if D['close'][i3] < D['open'][i5]:
                                    if (D['close'][i2] - D['open'][i2]) > 0:
                                        if D['close'][i2] < D['open'][i5]:
                                            if (D['close'][i1] - D['open'][i1]) < 0:
                                                if READING[i1] == 'MARIBOZU':
                                                    Read3 = 'FALLING THREE'
                                                elif READING[i1] == 'TUBE':
                                                    Read3 = 'FALLING THREE'
                                                else:
                                                    Read3 = 0
                                            else:
                                                Read3 = 0
                                        else:
                                            Read3 = 0
                                    else:
                                        Read3 = 0
                                else:
                                    Read3 = 0                               
                            else:
                                Read3 = 0
                        else:
                            Read3 = 0
                    else:
                        Read3 = 0
                else:
                    Read3 = 0
            else:
                Read3 = 0

            i1 = -1
            i2 = -2
            i3 = -3  
            i4 = -4  
            i5 = -5
            i6 = -6
            i7 = -7
            if (D['close'][i7] - D['open'][i7]) > 0:
                if READING[i7] == 'MARIBOZU':
                    if (D['close'][i6] - D['open'][i6]) < 0:
                        if D['close'][i6] > D['open'][i5]:
                            if (D['close'][i5] - D['open'][i5]) < 0:
                                if D['close'][i5] > D['open'][i5]:
                                    if (D['close'][i4] - D['open'][i4]) < 0:
                                        if D['close'][i4] > D['open'][i5]:
                                            if D['close'][i3] > D['open'][i5]:
                                                if D['close'][i2] > D['open'][i5]:
                                                    if (D['close'][i1] - D['open'][i1]) > 0:
                                                        if READING[i1] == 'MARIBOZU':
                                                            Read4 = 'MASTER CANDLE UP'
                                                        elif READING[i1] == 'TUBE':
                                                            Read4 = 'MASTER CANDLE UP'
                                                        else:
                                                            Read4 = 0
                                                    else:
                                                        Read4 = 0
                                                else:
                                                    Read4 = 0
                                            else:
                                                Read4 = 0
                                        else:
                                            Read4 = 0
                                    else:
                                        Read4 = 0
                                else:
                                    Read4 = 0
                            else:
                                Read4 = 0
                        else:
                            Read4 = 0
                    else:
                        Read4 = 0
                elif READING[i7] == 'TUBE':
                    if (D['close'][i6] - D['open'][i6]) < 0:
                        if D['close'][i6] > D['open'][i5]:
                            if (D['close'][i5] - D['open'][i5]) < 0:
                                if D['close'][i5] > D['open'][i5]:
                                    if (D['close'][i4] - D['open'][i4]) < 0:
                                        if D['close'][i4] > D['open'][i5]:
                                            if D['close'][i3] > D['open'][i5]:
                                                if D['close'][i2] > D['open'][i5]:
                                                    if (D['close'][i1] - D['open'][i1]) > 0:
                                                        if READING[i1] == 'MARIBOZU':
                                                            Read4 = 'MASTER CANDLE UP'
                                                        elif READING[i1] == 'TUBE':
                                                            Read4 = 'MASTER CANDLE UP'
                                                        else:
                                                            Read4 = 0
                                                    else:
                                                        Read4 = 0
                                                else:
                                                    Read4 = 0
                                            else:
                                                Read4 = 0
                                        else:
                                            Read4 = 0
                                    else:
                                        Read4 = 0
                                else:
                                    Read4 = 0
                            else:
                                Read4 = 0
                        else:
                            Read4 = 0
                    else:
                        Read4 = 0
                else:
                    Read4 = 0
            elif (D['close'][i7] - D['open'][i7]) < 0:
                if READING[i7] == 'MARIBOZU':
                    if (D['close'][i6] - D['open'][i6]) > 0:
                        if D['close'][i6] < D['open'][i5]:
                            if (D['close'][i5] - D['open'][i5]) > 0:
                                if D['close'][i5] < D['open'][i5]:
                                    if (D['close'][i4] - D['open'][i4]) > 0:
                                        if D['close'][i4] < D['open'][i5]:
                                            if D['close'][i3] < D['open'][i5]:
                                                if D['close'][i2] < D['open'][i5]:
                                                    if (D['close'][i1] - D['open'][i1]) < 0:
                                                        if READING[i1] == 'MARIBOZU':
                                                            Read4 = 'MASTER CANDLE DOWN'
                                                        elif READING[i1] == 'TUBE':
                                                            Read4 = 'MASTER CANDLE DOWN'
                                                        else:
                                                            Read4 = 0
                                                    else:
                                                        Read4 = 0
                                                else:
                                                    Read4 = 0
                                            else:
                                                Read4 = 0
                                        else:
                                            Read4 = 0
                                    else:
                                        Read4 = 0
                                else:
                                    Read4 = 0
                            else:
                                Read4 = 0
                        else:
                            Read4 = 0
                    else:
                        Read4 = 0
                elif READING[i7] == 'TUBE':
                    if (D['close'][i6] - D['open'][i6]) > 0:
                        if D['close'][i6] < D['open'][i5]:
                            if (D['close'][i5] - D['open'][i5]) > 0:
                                if D['close'][i5] < D['open'][i5]:
                                    if (D['close'][i4] - D['open'][i4]) > 0:
                                        if D['close'][i4] < D['open'][i5]:
                                            if D['close'][i3] < D['open'][i5]:
                                                if D['close'][i2] < D['open'][i5]:
                                                    if (D['close'][i1] - D['open'][i1]) < 0:
                                                        if READING[i1] == 'MARIBOZU':
                                                            Read4 = 'MASTER CANDLE DOWN'
                                                        elif READING[i1] == 'TUBE':
                                                            Read4 = 'MASTER CANDLE DOWN'
                                                        else:
                                                            Read4 = 0
                                                    else:
                                                        Read4 = 0
                                                else:
                                                    Read4 = 0
                                            else:
                                                Read4 = 0
                                        else:
                                            Read4 = 0
                                    else:
                                        Read4 = 0
                                else:
                                    Read4 = 0
                            else:
                                Read4 = 0
                        else:
                            Read4 = 0
                    else:
                        Read4 = 0
                else:
                    Read4 = 0
            else:
                Read4 = 0

            trade = 0
            if ST_slope > 0:
                if Read4 == 'MASTER CANDLE UP':
                    trade = 1
                elif Read3 == 'RISING THREE':
                    trade = 1
                elif READING2 == 'BULLISH ENGULFING':
                    trade = 1
                elif READING2 == 'TWEEZERS BOTTOM':
                    trade = 1
                elif READING2 == 'PIERCING PATTERN':
                    trade = 1
                elif READING == 'HAMMER':
                    trade = 1
                elif READING == 'DRAGONFLY DOJI':
                    trade = 1
            elif ST_slope < 0:
                if Read4 == 'MASTER CANDLE DOWN':
                    trade = -1
                elif Read3 == 'FALLING THREE':
                    trade = -1
                elif READING2 == 'BEARISH ENGULFING':
                    trade = -1
                elif READING2 == 'TWEEZERS TOP':
                    trade = -1
                elif READING2 == 'DARK CLOUD COVER':
                    trade = -1
                elif READING == 'SHOOTING STAR':
                    trade = -1
                elif READING == 'GRAVESTONE DOJI':
                    trade = -1

            entry = G['close'][-1]
            levels = levslist[ia]
            lastone = abs(levels[0] - entry)
            q = 0
            for x in levels:
                if q == 0:
                    Q = q
                leveldiff = abs(x - entry)
                if leveldiff < lastone:
                    Q = q
                lastone = leveldiff
                q += 1
            CTESTLVL = levels[Q]

            if trade == 1:
                position = "LONG"
            elif trade == -1:
                position = "SHORT"
            else:
                position = 0

            if position != 0:

                cend = int(len(levels)-1)
                if position == 'LONG':
                    clindex = levels.index(CTESTLVL)
                    tpindex = int(clindex - 1)
                    slindex = int(clindex + 1)
                    if clindex == 0:
                        stoploss = levels[slindex]
                        sldist = entry - stoploss
                        takeprofit = entry + sldist
                    elif clindex == cend:
                        takeprofit = levels[tpindex]
                        tpdist = takeprofit - entry
                        stoploss = entry - tpdist
                    else:
                        takeprofit = levels[tpindex]
                        stoploss = levels[slindex]
                elif position == 'SHORT':
                    clindex = levels.index(CTESTLVL)
                    tpindex = int(clindex + 1)
                    slindex = int(clindex - 1)
                    if clindex == 0:
                        takeprofit = levels[tpindex]
                        tpdist = entry - takeprofit
                        stoploss = entry + tpdist
                    elif clindex == cend:
                        stoploss = levels[slindex]
                        sldist = stoploss - entry
                        takeprofit = entry - sldist
                    else:
                        takeprofit = levels[tpindex]
                        stoploss = levels[slindex]
                else:
                    takeprofit = 0
                    stoploss = 0

                shrink = 0.90
                if position == 'LONG':
                    tpdist = takeprofit - entry
                    takeprofit = entry + tpdist*shrink
                elif position == 'SHORT':
                    tpdist = entry - takeprofit
                    takeprofit = entry - tpdist*shrink

                if position == 'LONG':
                    tpdist = takeprofit - entry
                    sldist = entry - stoploss
                    if sldist > tpdist:
                        stoploss = entry - tpdist
                elif position == 'SHORT':
                    tpdist = entry - takeprofit
                    sldist = stoploss - entry
                    if sldist > tpdist:
                        stoploss = entry + tpdist

                if trade == 1:
                    request, lotflag = positioner(p,"LONG",entry,takeprofit,stoploss)
                    if lotflag == "GREEN":
                        result = mt5.order_send(request)
                        if result.retcode != mt5.TRADE_RETCODE_DONE:
                            print("order_send failed, retcode={}".format(result.retcode))
                            result_dict=result._asdict()
                            for field in result_dict.keys():
                                print("   {}={}".format(field,result_dict[field]))
                                if field=="request":
                                    traderequest_dict=result_dict[field]._asdict()
                                    for tradereq_filed in traderequest_dict:
                                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                        else:
                            print("Trade Placed:   ",p,"   ","Long","   TP:  ",takeprofit,"   SL:  ",stoploss,)
                elif trade == -1:
                    request, lotflag = positioner(p,"SHORT",entry,takeprofit,stoploss)
                    if lotflag == "GREEN":
                        result = mt5.order_send(request)
                        if result.retcode != mt5.TRADE_RETCODE_DONE:
                            print("order_send failed, retcode={}".format(result.retcode))
                            result_dict=result._asdict()
                            for field in result_dict.keys():
                                print("   {}={}".format(field,result_dict[field]))
                                if field=="request":
                                    traderequest_dict=result_dict[field]._asdict()
                                    for tradereq_filed in traderequest_dict:
                                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                        else:
                            print("Trade Placed:   ",p,"   ","Short","   TP:  ",takeprofit,"   SL:  ",stoploss,)
                print(position,entry,takeprofit,stoploss,lotflag)
        ia += 1

In [11]:
print("Initializing...")
pair = []
symbols=mt5.symbols_get()
for s in symbols:
    pair.append(s.name)
    
levslist = levelgrab(pair)
    
t2 = 0
while True:
    Drates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M5, 0, 1)
    D = Drates[0]
    t = D[0]
    if t2 != 0:
        if t != t2:
            print("Analyzing Markets...")
            RUBY(pair, levslist)
    t2 = t
    time.sleep(1)

Initializing...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
order_send failed, retcode=10044
   retcode=10044
   deal=0
   order=0
   volume=0.0
   price=0.0
   bid=0.0
   ask=0.0
   comment=Only position closing is allowed
   request_id=0
   retcode_external=0
   request=TradeRequest(action=1, magic=234000, order=0, symbol='USDTRY', volume=0.25, price=6.95605, stoplimit=0.0, sl=6.969885, tp=6.933545499999999, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='RUBY', position=0, position_by=0)
       traderequest: action=1
       traderequest: magic=234000
       traderequest: order=0
       traderequest: symbol=USDTRY
       traderequest: volume=0.25
       traderequest: price=6.95605
       traderequest: stoplimit=0.0
       traderequest: sl=6.969885
       traderequest: tp=6.933545499999999
       traderequest: deviation=20
       traderequest: type=1
       traderequest: type_fi

KeyboardInterrupt: 